# Construcción de un Modelo Generativo de Lenguaje con Naive Bayes

Naive Bayes es un algoritmo basado en la probabilidad condicional que asume independencia entre las características. En el contexto de los modelos de lenguaje, se utiliza para predecir la probabilidad de una palabra en una secuencia dada, basándose en el conteo de frecuencias.

---

### 1. El Teorema de Bayes

El Teorema de Bayes describe la relación entre la probabilidad condicional de dos eventos. Matemáticamente, se expresa como:

$$
P(A \mid B) = \frac{P(B \mid A) P(A)}{P(B)}
$$

Donde:
- \( P(A | B) \) es la probabilidad posterior, es decir, la probabilidad de que ocurra el evento \( A \) dado que ha ocurrido el evento \( B \).
- \( P(B | A) \) es la probabilidad condicional de \( B \) dado \( A \).
- \( P(A) \) es la probabilidad previa de \( A \) (la probabilidad de que ocurra \( A \) sin información adicional).
- \( P(B) \) es la probabilidad marginal de \( B \).

Este teorema es la base de los clasificadores de Naive Bayes.

---



### 2. Fórmula General de Naive Bayes en un contexto de Modelo de Lenguaje
Dada una secuencia de palabras , queremos predecir la probabilidad de la secuencia completa:

$$
P(w_1, w_2, \dots, w_n) = P(w_1) P(w_2 | w_1) P(w_3 | w_1, w_2) \dots P(w_n | w_1, w_2, \dots, w_{n-1})
$$


Naive Bayes simplifica este cálculo asumiendo que cada palabra depende únicamente de la palabra anterior, lo que da lugar al modelo de n-gramas.
Para el caso de bigramas:

$$
P(w_1, w_2, \dots, w_n) = P(w_1) P(w_2 | w_1) P(w_3 | w_2) \dots P(w_n | w_{n-1})
$$

In [32]:
import re
# Definimos nuestro tokenizador a palabras

def tokenizer(corpus:str)->list:
    # Convertimos a minuscula
    new_corpus = corpus.lower()
    pattern = re.compile(r'\b\w+\b')
    return re.findall(pattern, new_corpus)

corpus = "Como estas te veo bien por ahora"
tokenizer(corpus)

['como', 'estas', 'te', 'veo', 'bien', 'por', 'ahora']

In [33]:
import re

pattern = re.compile(r'\b\w+\b|[.,?"]')
re.findall(pattern,'hola como estas')

['hola', 'como', 'estas']

In [34]:
from collections import defaultdict

# Definimos una funcion que cuenta los unigramas, bigramas, trigramas y cuatrigramas
def get_unigrams_bigrams_trigramas_cuatrigramas(corpus_tokenized: list) -> (dict, dict, dict, dict):
    # Usamos defaultdict para evitar tener que inicializar las claves manualmente
    unigrams = defaultdict(int)
    bigrams = defaultdict(int)
    trigramas = defaultdict(int)
    cuatrigramas = defaultdict(int)

    n = len(corpus_tokenized)

    if n < 4 :
        return None,None,None,None

    for i in range(n - 3):
        unigram = corpus_tokenized[i]
        bigram = (corpus_tokenized[i], corpus_tokenized[i + 1])
        trigrama = (corpus_tokenized[i], corpus_tokenized[i + 1], corpus_tokenized[i + 2])
        cuatrigrama = (corpus_tokenized[i], corpus_tokenized[i + 1], corpus_tokenized[i + 2], corpus_tokenized[i + 3])

        # Los contadores se incrementan automáticamente
        unigrams[unigram] += 1
        bigrams[bigram] += 1
        trigramas[trigrama] += 1
        cuatrigramas[cuatrigrama] += 1


    # Agregamos los 3 ultimos unigramas
    unigrams[corpus_tokenized[n - 3]] += 1
    unigrams[corpus_tokenized[n - 2]] += 1
    unigrams[corpus_tokenized[n - 1]] += 1
    # Agregamos los 2 ultimos bigramas
    bigrams[(corpus_tokenized[n - 3],corpus_tokenized[n - 2])] += 1
    bigrams[(corpus_tokenized[n - 2],corpus_tokenized[n - 1])] += 1

    # Agregamos el ultimo trigrama
    trigrama = (corpus_tokenized[n - 3],corpus_tokenized[n - 2],corpus_tokenized[n - 1])
    trigramas[trigrama] += 1

    return dict(unigrams), dict(bigrams), dict(trigramas), dict(cuatrigramas)


corpus = """
El Misterio del Bosque Encantado

Una fría tarde de otoño, Elena decidió explorar el viejo bosque que se encontraba a las afueras del pueblo. La gente solía decir que aquel lugar estaba maldito, que quienes se aventuraban demasiado en sus profundidades no regresaban jamás. Pero a Elena no le importaban esas historias; tenía una curiosidad insaciable y un espíritu aventurero que no conocía el miedo.
Al adentrarse en el bosque, la luz del sol comenzó a desvanecerse entre los árboles, creando sombras alargadas que parecían moverse a su alrededor. A cada paso, las hojas secas crujían bajo sus botas, resonando en la inquietante tranquilidad que envolvía el lugar. Todo estaba en silencio, excepto por el ocasional canto de un cuervo en lo alto.
De repente, a lo lejos, divisó una figura. Parecía un niño, inmóvil, con una capa oscura que se confundía con los troncos de los árboles. Elena, intrigada, decidió acercarse. Cuanto más se aproximaba, más claro veía que el niño no parecía estar del todo presente, como si fuera parte del mismo bosque, una extensión más de sus secretos. Al llegar junto a él, sus ojos, de un azul profundo, se encontraron con los de Elena.
—No deberías estar aquí —dijo el niño con una voz suave, casi susurrante.
Elena sintió un escalofrío recorrer su espalda, pero antes de poder responder, la figura desapareció en el aire como si nunca hubiera estado allí. Confusa y con el corazón acelerado, miró a su alrededor, pero el bosque estaba desierto.
Decidió regresar al pueblo, pero el camino que había tomado parecía haber cambiado. Las mismas rutas que antes le resultaban familiares ahora parecían distorsionadas, llevándola en círculos sin un destino claro. El sol se estaba poniendo, y la oscuridad envolvía el lugar cada vez más. Respirando profundamente, Elena intentó calmarse, pero no podía dejar de pensar en las palabras del niño.
Finalmente, después de lo que le pareció una eternidad, encontró la salida. Al llegar al pueblo, notó que algo había cambiado. Las casas lucían más deterioradas, como si hubieran pasado años desde que las había visto por última vez. Al preguntar a los aldeanos, descubrió con horror que había desaparecido por tres días, aunque para ella solo había pasado una tarde.
El bosque encantado había jugado con el tiempo una vez más, dejando a Elena con más preguntas que respuestas y una advertencia clara en su mente: nunca volvería a entrar.
"""

unigrams,bigrams,trigrams,cuatrigrams = get_unigrams_bigrams_trigramas_cuatrigramas(tokenizer(corpus))

In [35]:
unigrams

{'el': 16,
 'misterio': 1,
 'del': 6,
 'bosque': 6,
 'encantado': 2,
 'una': 10,
 'fría': 1,
 'tarde': 2,
 'de': 10,
 'otoño': 1,
 'elena': 7,
 'decidió': 3,
 'explorar': 1,
 'viejo': 1,
 'que': 15,
 'se': 6,
 'encontraba': 1,
 'a': 11,
 'las': 6,
 'afueras': 1,
 'pueblo': 3,
 'la': 6,
 'gente': 1,
 'solía': 1,
 'decir': 1,
 'aquel': 1,
 'lugar': 3,
 'estaba': 4,
 'maldito': 1,
 'quienes': 1,
 'aventuraban': 1,
 'demasiado': 1,
 'en': 9,
 'sus': 4,
 'profundidades': 1,
 'no': 6,
 'regresaban': 1,
 'jamás': 1,
 'pero': 5,
 'le': 3,
 'importaban': 1,
 'esas': 1,
 'historias': 1,
 'tenía': 1,
 'curiosidad': 1,
 'insaciable': 1,
 'y': 4,
 'un': 6,
 'espíritu': 1,
 'aventurero': 1,
 'conocía': 1,
 'miedo': 1,
 'al': 6,
 'adentrarse': 1,
 'luz': 1,
 'sol': 2,
 'comenzó': 1,
 'desvanecerse': 1,
 'entre': 1,
 'los': 5,
 'árboles': 2,
 'creando': 1,
 'sombras': 1,
 'alargadas': 1,
 'parecían': 2,
 'moverse': 1,
 'su': 4,
 'alrededor': 2,
 'cada': 2,
 'paso': 1,
 'hojas': 1,
 'secas': 1,
 'crují

In [36]:
bigrams

{('el', 'misterio'): 1,
 ('misterio', 'del'): 1,
 ('del', 'bosque'): 1,
 ('bosque', 'encantado'): 2,
 ('encantado', 'una'): 1,
 ('una', 'fría'): 1,
 ('fría', 'tarde'): 1,
 ('tarde', 'de'): 1,
 ('de', 'otoño'): 1,
 ('otoño', 'elena'): 1,
 ('elena', 'decidió'): 1,
 ('decidió', 'explorar'): 1,
 ('explorar', 'el'): 1,
 ('el', 'viejo'): 1,
 ('viejo', 'bosque'): 1,
 ('bosque', 'que'): 1,
 ('que', 'se'): 2,
 ('se', 'encontraba'): 1,
 ('encontraba', 'a'): 1,
 ('a', 'las'): 1,
 ('las', 'afueras'): 1,
 ('afueras', 'del'): 1,
 ('del', 'pueblo'): 1,
 ('pueblo', 'la'): 1,
 ('la', 'gente'): 1,
 ('gente', 'solía'): 1,
 ('solía', 'decir'): 1,
 ('decir', 'que'): 1,
 ('que', 'aquel'): 1,
 ('aquel', 'lugar'): 1,
 ('lugar', 'estaba'): 1,
 ('estaba', 'maldito'): 1,
 ('maldito', 'que'): 1,
 ('que', 'quienes'): 1,
 ('quienes', 'se'): 1,
 ('se', 'aventuraban'): 1,
 ('aventuraban', 'demasiado'): 1,
 ('demasiado', 'en'): 1,
 ('en', 'sus'): 1,
 ('sus', 'profundidades'): 1,
 ('profundidades', 'no'): 1,
 ('no', 'r

In [37]:
trigrams

{('el', 'misterio', 'del'): 1,
 ('misterio', 'del', 'bosque'): 1,
 ('del', 'bosque', 'encantado'): 1,
 ('bosque', 'encantado', 'una'): 1,
 ('encantado', 'una', 'fría'): 1,
 ('una', 'fría', 'tarde'): 1,
 ('fría', 'tarde', 'de'): 1,
 ('tarde', 'de', 'otoño'): 1,
 ('de', 'otoño', 'elena'): 1,
 ('otoño', 'elena', 'decidió'): 1,
 ('elena', 'decidió', 'explorar'): 1,
 ('decidió', 'explorar', 'el'): 1,
 ('explorar', 'el', 'viejo'): 1,
 ('el', 'viejo', 'bosque'): 1,
 ('viejo', 'bosque', 'que'): 1,
 ('bosque', 'que', 'se'): 1,
 ('que', 'se', 'encontraba'): 1,
 ('se', 'encontraba', 'a'): 1,
 ('encontraba', 'a', 'las'): 1,
 ('a', 'las', 'afueras'): 1,
 ('las', 'afueras', 'del'): 1,
 ('afueras', 'del', 'pueblo'): 1,
 ('del', 'pueblo', 'la'): 1,
 ('pueblo', 'la', 'gente'): 1,
 ('la', 'gente', 'solía'): 1,
 ('gente', 'solía', 'decir'): 1,
 ('solía', 'decir', 'que'): 1,
 ('decir', 'que', 'aquel'): 1,
 ('que', 'aquel', 'lugar'): 1,
 ('aquel', 'lugar', 'estaba'): 1,
 ('lugar', 'estaba', 'maldito'): 1,


In [38]:
cuatrigrams

{('el', 'misterio', 'del', 'bosque'): 1,
 ('misterio', 'del', 'bosque', 'encantado'): 1,
 ('del', 'bosque', 'encantado', 'una'): 1,
 ('bosque', 'encantado', 'una', 'fría'): 1,
 ('encantado', 'una', 'fría', 'tarde'): 1,
 ('una', 'fría', 'tarde', 'de'): 1,
 ('fría', 'tarde', 'de', 'otoño'): 1,
 ('tarde', 'de', 'otoño', 'elena'): 1,
 ('de', 'otoño', 'elena', 'decidió'): 1,
 ('otoño', 'elena', 'decidió', 'explorar'): 1,
 ('elena', 'decidió', 'explorar', 'el'): 1,
 ('decidió', 'explorar', 'el', 'viejo'): 1,
 ('explorar', 'el', 'viejo', 'bosque'): 1,
 ('el', 'viejo', 'bosque', 'que'): 1,
 ('viejo', 'bosque', 'que', 'se'): 1,
 ('bosque', 'que', 'se', 'encontraba'): 1,
 ('que', 'se', 'encontraba', 'a'): 1,
 ('se', 'encontraba', 'a', 'las'): 1,
 ('encontraba', 'a', 'las', 'afueras'): 1,
 ('a', 'las', 'afueras', 'del'): 1,
 ('las', 'afueras', 'del', 'pueblo'): 1,
 ('afueras', 'del', 'pueblo', 'la'): 1,
 ('del', 'pueblo', 'la', 'gente'): 1,
 ('pueblo', 'la', 'gente', 'solía'): 1,
 ('la', 'gente',

### Probabilidades de Unigrama, Bigrama, Trigrama y Cuatrigrama

#### Probabilidad de un Unigrama

La fórmula para calcular la probabilidad de un unigrama \( $P(w_1)$ \) es la siguiente:

$$
P(w_1) = \frac{\text{frecuencia}(w_1)}{N}
$$

Donde:

- \( $\text{frecuencia}(w_1)$ \) es el número de veces que la palabra \( $w_1$ \) aparece en el corpus.
- \( N \) es el número total de ocurrencias de los unigramas

Esta fórmula calcula la probabilidad de que un unigrama \( $w_1$ \) ocurra en el conjunto de datos, basado en su frecuencia relativa.


In [44]:
# PROBABILIDADES CONDICIONALES USANDO NAIVE BAYES CON SUAVIZADO
def get_prob_unigrams(unigrams: dict) -> dict:
    unigrams_prob = {}
    total_count = sum(unigrams.values())  # Total de ocurrencias de unigramas
    for unigram, freq in unigrams.items():
        unigrams_prob[unigram] = freq / total_count  # P(w) = freq(w) / total unigrams
    return unigrams_prob

#### Probabilidad de un Bigrama

La fórmula para calcular la probabilidad condicional de un bigrama \( $P(w_2 \mid w_1)$ \) con suavizado de Laplace es la siguiente:

$$
P(w_2 \mid w_1) = \frac{\text{frecuencia}(w_1, w_2) + \alpha}{\text{frecuencia}(w_1) + \alpha \cdot |V|}
$$

Donde:

- \( $\text{frecuencia}(w_1, w_2)$ \) es el número de veces que el bigrama \( $(w_1, w_2)$ \) aparece en el corpus.
- \( $\text{frecuencia}(w_1)$ \) es el número de veces que la palabra \( $w_1$ \) aparece en el corpus.
- \( $\alpha$ \) es el parámetro de suavizado, comúnmente \( $\alpha = 1$ \) en el suavizado de Laplace.
- \( $|V|$ \) es el tamaño del vocabulario, es decir, el número de palabras únicas en el corpus.

El suavizado de Laplace ayuda a evitar probabilidades cero para bigramas no observados en el corpus.


In [45]:
def get_prob_bigrams(bigrams: dict, unigrams: dict, alpha=0.1) -> dict:
    bigrams_prob = {}
    V = len(unigrams)  # Tamaño del vocabulario (suavizado Laplace)
    for bigram, freq in bigrams.items():
        bigrams_prob[bigram] = (freq + alpha) / (unigrams[bigram[0]] + V * alpha)  # P(w2|w1)
    return bigrams_prob

#### Probabilidad de un Trigrama

La fórmula para calcular la probabilidad condicional de un trigrama \( $P(w_3 \mid w_1, w_2)$ \) usando suavizado de Laplace es la siguiente:

$$
P(w_3 \mid w_1, w_2) = \frac{\text{frecuencia}(w_1, w_2, w_3) + \alpha}{\text{frecuencia}(w_1, w_2) + \alpha \cdot |V|}
$$

Donde:

- \( $\text{frecuencia}(w_1, w_2, w_3)$ \) es el número de veces que el trigrama \( $(w_1, w_2, w_3)$ \) aparece en el corpus.
- \( $\text{frecuencia}(w_1, w_2)$ \) es el número de veces que el bigrama \( $(w_1, w_2)$ \) aparece en el corpus.
- \( $\alpha$ \) es el parámetro de suavizado.
- \( $|V|$ \) es el tamaño del vocabulario de bigramas.

El suavizado se utiliza para ajustar las probabilidades y evitar problemas con trigramas que no se encuentran en los datos.


In [46]:
def get_prob_trigrams(trigramas: dict, bigrams: dict, alpha=0.1) -> dict:
    trigramas_prob = {}
    V = len(bigrams)  # Usamos el número de bigramas para el suavizado
    for trigrama, freq in trigramas.items():
        bigram = (trigrama[0], trigrama[1])
        trigramas_prob[trigrama] = (freq + alpha) / (bigrams[bigram] + V * alpha)  # P(w3|w1,w2)
    return trigramas_prob

    

#### Probabilidad de un Cuatrigrama

La fórmula para calcular la probabilidad condicional de un cuatrigrama \( $P(w_4 \mid w_1, w_2, w_3)$ \) es la siguiente:

$$
P(w_4 \mid w_1, w_2, w_3) = \frac{\text{frecuencia}(w_1, w_2, w_3, w_4) + \alpha}{\text{frecuencia}(w_1, w_2, w_3) + \alpha \cdot |V|}
$$

Donde:

- \( $\text{frecuencia}(w_1, w_2, w_3, w_4)$ \) es el número de veces que el cuatrigrama \( $(w_1, w_2, w_3, w_4)$ \) aparece en el corpus.
- \( $\text{frecuencia}(w_1, w_2, w_3)$ \) es el número de veces que el trigrama \( $(w_1, w_2, w_3)$ \) aparece en el corpus.
- \( $\alpha$ \) es el parámetro de suavizado.
- \( $|V|$ \) es el tamaño del vocabulario de trigramas.

El suavizado de Laplace ayuda a asegurar que se calculen probabilidades no nulas incluso para cuatrigramas raros o no observados en el corpus.

In [47]:
def get_prob_cuatrigramas(cuatrigramas: dict, trigramas: dict, alpha=0.1) -> dict:
    cuatrigramas_prob = {}
    V = len(trigramas)  # Suavizado usando trigramas
    for cuatrigrama, freq in cuatrigramas.items():
        trigrama = (cuatrigrama[0], cuatrigrama[1], cuatrigrama[2])
        cuatrigramas_prob[cuatrigrama] = (freq + alpha) / (trigramas[trigrama] + V * alpha)  # P(w4|w1,w2,w3)
    return cuatrigramas_prob

In [50]:
unigrams_prob , bigrams_prob =  get_prob_unigrams(unigrams) , get_prob_bigrams(bigrams,unigrams)
trigrams_prob , cuatrigrams_prob = get_prob_trigrams(trigrams,bigrams), get_prob_cuatrigramas(cuatrigrams,trigrams)

In [57]:
print('Top 5 unigramas mas probables:')
sorted(unigrams_prob.items(),key=lambda x : x[1], reverse=True)[:5]

Top 5 unigramas mas probables:


[('el', 0.03980099502487562),
 ('que', 0.03731343283582089),
 ('a', 0.02736318407960199),
 ('una', 0.024875621890547265),
 ('de', 0.024875621890547265)]

In [58]:
print('Top 5 bigramas mas probables:')
sorted(bigrams_prob.items(),key=lambda x : x[1], reverse=True)[:5]

Top 5 bigramas mas probables:


[(('como', 'si'), 0.12757201646090535),
 (('envolvía', 'el'), 0.09012875536480687),
 (('cambiado', 'las'), 0.09012875536480687),
 (('vez', 'más'), 0.08641975308641975),
 (('el', 'bosque'), 0.08310991957104559)]

In [60]:
print('Top 5 trigramas mas probables:')
sorted(trigrams_prob.items(),key=lambda x : x[1], reverse=True)[:5]

Top 5 trigramas mas probables:


[(('a', 'su', 'alrededor'), 0.05303030303030303),
 (('envolvía', 'el', 'lugar'), 0.05303030303030303),
 (('el', 'misterio', 'del'), 0.02849740932642487),
 (('misterio', 'del', 'bosque'), 0.02849740932642487),
 (('del', 'bosque', 'encantado'), 0.02849740932642487)]

In [61]:
print('Top 5 cuatrigramas mas probables:')
sorted(cuatrigrams_prob.items(),key=lambda x : x[1], reverse=True)[:5]

Top 5 cuatrigramas mas probables:


[(('el', 'misterio', 'del', 'bosque'), 0.026960784313725488),
 (('misterio', 'del', 'bosque', 'encantado'), 0.026960784313725488),
 (('del', 'bosque', 'encantado', 'una'), 0.026960784313725488),
 (('bosque', 'encantado', 'una', 'fría'), 0.026960784313725488),
 (('encantado', 'una', 'fría', 'tarde'), 0.026960784313725488)]

### Predicción de la siguiente palabra según un contexto

Para una ventana de \( s = 4 \), utilizamos un enfoque de **Naive Bayes** para combinar las probabilidades condicionales de unigramas, bigramas, trigramas y cuatrigramas con el fin de predecir la siguiente palabra en una secuencia.

#### Expresión matemática

La probabilidad de la siguiente palabra \( $w_4$ \) dado el contexto \( $w_1, w_2, w_3$ \) se puede calcular combinando las probabilidades condicionales de los n-gramas como sigue:

$$
P(w_4 \mid w_1, w_2, w_3) \propto P(w_4 \mid w_1, w_2, w_3) \cdot P(w_4 \mid w_2, w_3) \cdot P(w_4 \mid w_3) \cdot P(w_4)
$$

Donde:

- \( $P(w_4 \mid w_1, w_2, w_3)$ \) es la probabilidad condicional basada en los **cuatrigramas**.
- \( $P(w_4 \mid w_2, w_3)$ \) es la probabilidad condicional basada en los **trigramas**.
- \( $P(w_4 \mid w_3)$ \) es la probabilidad condicional basada en los **bigramas**.
- \( $P(w_4)$ \) es la probabilidad basada en los **unigramas**.

Cada una de estas probabilidades se suaviza utilizando el suavizado de Laplace para evitar probabilidades de cero cuando una secuencia no ha sido observada. La fórmula general para cada n-grama con suavizado de Laplace es la siguiente:


In [82]:
def predict_next_word_naive_bayes(w1, w2, w3, unigrams, bigrams, trigrams, cuatrigramas, alpha=0.1):
    # Lista de palabras candidatas (por simplicidad, usamos el vocabulario de unigramas)
    vocabulario = list(unigrams.keys())
    max_prob = 0
    best_word = None
    # Alamacenamos las probabilidades de cada palabra candidata
    probs = {}
    
    # Iterar por cada palabra candidata en el vocabulario
    for w4 in vocabulario:
        # Calcular las probabilidades individuales para cada n-grama
        # Cuatrigrama
        if (w1, w2, w3, w4) in cuatrigramas:
            p_cuatrigrama = (cuatrigramas[(w1, w2, w3, w4)] + alpha) / (trigrams[(w1, w2, w3)] + alpha * len(vocabulario))
        else:
            p_cuatrigrama = alpha / (trigrams.get((w1, w2, w3), 0) + alpha * len(vocabulario))

        # Trigrama
        if (w2, w3, w4) in trigrams:
            p_trigrama = (trigrams[(w2, w3, w4)] + alpha) / (bigrams[(w2, w3)] + alpha * len(vocabulario))
        else:
            p_trigrama = alpha / (bigrams.get((w2, w3), 0) + alpha * len(vocabulario))

        # Bigrama
        if (w3, w4) in bigrams:
            p_bigrama = (bigrams[(w3, w4)] + alpha) / (unigrams[w3] + alpha * len(vocabulario))
        else:
            p_bigrama = alpha / (unigrams.get(w3, 0) + alpha * len(vocabulario))

        # Unigrama (para suavizado en general)
        p_unigrama = (unigrams[w4] + alpha) / (sum(unigrams.values()) + alpha * len(vocabulario))

        # Calcular la probabilidad conjunta usando Naive Bayes (producto de las probabilidades)
        prob_total = p_cuatrigrama * p_trigrama * p_bigrama * p_unigrama
        #print(f'{w4} : {prob_total}')
        probs[w4] = prob_total

        # Mantener la palabra con la probabilidad más alta
        if prob_total > max_prob:
            max_prob = prob_total
            best_word = w4

    return best_word,probs

def predict_words(frase:str,unigrams:dict,bigrams:dict,trigrams:dict,cuatrigrams:dict, k:int,alpha=0.1):
    words = tokenizer(frase)
    n = len(words)
    if n < 3:
        return None
    w1 = words[-3]
    w2 = words[-2]
    w3 = words[-1]
    new_words = []
    for i in range(k):
        new_word,probs = predict_next_word_naive_bayes(w1,w2,w3,unigrams,bigrams,trigrams,cuatrigrams,alpha)
        # Imprimir la palabra predicha y su probabilidad   
        print(f'{w1} {w2} {w3} {new_word} : {probs[new_word]}')

        # Actualizar las palabras para la siguiente iteración
        w1 = w2
        w2 = w3
        w3 = new_word

        
        new_words.append(new_word)
    return new_words


# Ejemplo de uso
w1 = 'troncos'
w2 = 'de'
w3 = 'los'
frase = w1 + ' ' + w2 + ' ' + w3
new_words,probs = predict_next_word_naive_bayes(w1,w2,w3,unigrams, bigrams, trigrams, cuatrigrams)
print(f"Siguiente palabra: {frase} -> ", new_words)
print(sorted(probs.items(),key=lambda x : x[1], reverse=True)[:5])

# predecir las siguientes 5 palabras
frase = 'the human is'
predict_words(frase,unigrams,bigrams,trigrams,cuatrigrams,5,alpha=0.1)

Siguiente palabra: troncos de los ->  the
[('the', 3.4580450139140344e-13), ('of', 1.5872695243526675e-13), ('and', 1.5573028689134696e-13), ('to', 1.5530219181364413e-13), ('i', 1.5508814427479272e-13)]
the human is the : 5.995071190146243e-11
human is the one : 1.3006308809907175e-11
is the one which : 3.139606256027356e-10
the one which is : 1.3420561807506389e-09
one which is the : 1.7866139659432056e-09


['the', 'one', 'which', 'is', 'the']

In [71]:
# Lectura del corpus
path_corpus = './corpus/arthur-conan-doyle.tok.test.txt'

def leer_corpus(archivo):
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        # Si utf-8 falla, intentamos con latin-1
        with open(archivo, 'r', encoding='latin-1') as f:
            return f.read()

# Leer el archivo
corpus = leer_corpus(path_corpus)
# Imprimimos un extracto del corpus
print("\nExtracto del corpus:")
print(corpus[:2000])
# Imprimimos la longitud del corpus
print("\nLongitud del corpus:", len(corpus))

unigrams,bigrams,trigrams,cuatrigrams = get_unigrams_bigrams_trigramas_cuatrigramas(tokenizer(corpus))

unigrams_prob , bigrams_prob =  get_prob_unigrams(unigrams) , get_prob_bigrams(bigrams,unigrams)
trigrams_prob , cuatrigrams_prob = get_prob_trigrams(trigrams,bigrams), get_prob_cuatrigramas(cuatrigrams,trigrams)


Extracto del corpus:
VI . The Musgrave Ritual
An anomaly which often struck me in the character of my friend Sherlock Holmes was that , although in his methods of thought he was the neatest and most methodical of mankind , and although also he affected a certain quiet primness of dress , he was none the less in his personal habits one of the most untidy men that ever drove a fellow - lodger to distraction . Not that I am in the least conventional in that respect myself . The rough - and - tumble work in Afghanistan , coming on the top of a natural Bohemianism of disposition , has made me rather more lax than befits a medical man . But with me there is a limit , and when I find a man who keeps his cigars in the coal - scuttle , his tobacco in the toe end of a Persian slipper , and his unanswered correspondence transfixed by a jack - knife into the very centre of his wooden mantelpiece , then I begin to give myself virtuous airs . I have always held , too , that pistol practice should b

In [75]:
# Ejemplo de uso
w1 = 'the'
w2 = 'human'
w3 = 'is'
frase = w1 + ' ' + w2 + ' ' + w3
new_words,probs = predict_next_word_naive_bayes(w1,w2,w3,unigrams, bigrams, trigrams, cuatrigrams)
print(f"Siguiente palabra: {frase} -> ", new_words)
sorted(probs.items(),key=lambda x : x[1], reverse=True)[:5]

Siguiente palabra: the human is ->  the


[('the', 5.995071190146243e-11),
 ('a', 4.3329334381382753e-11),
 ('of', 1.255345683906054e-11),
 ('to', 7.402095534383316e-12),
 ('that', 4.8566846931888605e-12)]

# Constuyendo la clase Generadora

In [154]:
import re
from collections import defaultdict
import numpy as np
import math

class NaiveBayesNGramModel:
    def __init__(self, corpus_path: str):
        self.corpus_path = corpus_path
        self.corpus = self.leer_corpus()
        self.corpus_tokenized = self.tokenizer(self.corpus)

        # Inicializamos unigramas, bigramas, trigramas y cuatrigramas
        self.unigrams, self.bigrams, self.trigrams, self.cuatrigramas = self.get_unigrams_bigrams_trigramas_cuatrigramas(self.corpus_tokenized)

        # Calculamos las probabilidades de n-gramas con enfoque Naive Bayes
        self.unigrams_prob = self.get_prob_unigrams(self.unigrams)
        self.bigrams_prob = self.get_prob_bigrams(self.bigrams, self.unigrams)
        self.trigrams_prob = self.get_prob_trigrams(self.trigrams,self.bigrams)
        self.cuatrigrams_prob = self.get_prob_cuatrigramas(self.cuatrigramas,self.trigrams)

    # Lectura del corpus desde archivo
    def leer_corpus(self):
        try:
            with open(self.corpus_path, 'r', encoding='utf-8') as f:
                return f.read()
        except UnicodeDecodeError:
            with open(self.corpus_path, 'r', encoding='latin-1') as f:
                return f.read()

    # Tokenizador
    def tokenizer(self, corpus: str) -> list:
        """
        Tokeniza el texto eliminando caracteres especiales, metadatos y convirtiendo a minúsculas.
        """
        # Eliminar metadatos como "Provincia:", "Fecha:", "Temas:", y etiquetas como [HS:].
        corpus_cleaned = re.sub(r"Provincia:.*|Enclave:.*|Fecha:.*|Duraci.*|Informantes:.*|Encuesta:.*|Transcripci.*|Temas:.*", "", corpus)
        
        # Eliminar anotaciones dentro de corchetes como [HS:E1], [RISA], etc.
        corpus_cleaned = re.sub(r"\[.*?\]", "", corpus_cleaned)

        # Eliminar múltiples espacios y líneas vacías
        corpus_cleaned = re.sub(r"\s+", " ", corpus_cleaned).strip()

        # Convertir a minúsculas y tokenizar por palabras alfanuméricas
        corpus_cleaned = corpus_cleaned.lower()
        pattern = re.compile(r'\b\w+\b')  # Tokenizar solo palabras alfanuméricas
        return re.findall(pattern, corpus_cleaned)

    # Función para obtener unigramas, bigramas, trigramas y cuatrigramas
    def get_unigrams_bigrams_trigramas_cuatrigramas(self, corpus_tokenized: list):
        unigrams = defaultdict(int)
        bigrams = defaultdict(int)
        trigramas = defaultdict(int)
        cuatrigramas = defaultdict(int)

        n = len(corpus_tokenized)
        if n < 4:
            return None, None, None, None

        for i in range(n - 3):
            unigram = corpus_tokenized[i]
            bigram = (corpus_tokenized[i], corpus_tokenized[i + 1])
            trigrama = (corpus_tokenized[i], corpus_tokenized[i + 1], corpus_tokenized[i + 2])
            cuatrigrama = (corpus_tokenized[i], corpus_tokenized[i + 1], corpus_tokenized[i + 2], corpus_tokenized[i + 3])

            unigrams[unigram] += 1
            bigrams[bigram] += 1
            trigramas[trigrama] += 1
            cuatrigramas[cuatrigrama] += 1

        # Agregamos los últimos n-gramas
        unigrams[corpus_tokenized[n - 3]] += 1
        unigrams[corpus_tokenized[n - 2]] += 1
        unigrams[corpus_tokenized[n - 1]] += 1
        bigrams[(corpus_tokenized[n - 3], corpus_tokenized[n - 2])] += 1
        bigrams[(corpus_tokenized[n - 2], corpus_tokenized[n - 1])] += 1
        trigramas[(corpus_tokenized[n - 3], corpus_tokenized[n - 2], corpus_tokenized[n - 1])] += 1

        return dict(unigrams), dict(bigrams), dict(trigramas), dict(cuatrigramas)

    # PROBABILIDADES CONDICIONALES USANDO NAIVE BAYES CON SUAVIZADO
    def get_prob_unigrams(self, unigrams: dict) -> dict:
        unigrams_prob = {}
        total_count = sum(unigrams.values())  # Total de ocurrencias de unigramas
        for unigram, freq in unigrams.items():
            unigrams_prob[unigram] = freq / total_count  # P(w) = freq(w) / total unigrams
        return unigrams_prob

    def get_prob_bigrams(self, bigrams: dict, unigrams: dict, alpha=0.1) -> dict:
        bigrams_prob = {}
        V = len(unigrams)  # Tamaño del vocabulario (suavizado Laplace)
        for bigram, freq in bigrams.items():
            bigrams_prob[bigram] = (freq + alpha) / (unigrams[bigram[0]] + V * alpha)  # P(w2|w1)
        return bigrams_prob

    def get_prob_trigrams(self, trigramas: dict, bigrams: dict, alpha=0.1) -> dict:
        trigramas_prob = {}
        V = len(bigrams)  # Usamos el número de bigramas para el suavizado
        for trigrama, freq in trigramas.items():
            bigram = (trigrama[0], trigrama[1])
            trigramas_prob[trigrama] = (freq + alpha) / (bigrams[bigram] + V * alpha)  # P(w3|w1,w2)
        return trigramas_prob

    def get_prob_cuatrigramas(self, cuatrigramas: dict, trigramas: dict, alpha=0.1) -> dict:
        cuatrigramas_prob = {}
        V = len(trigramas)  # Suavizado usando trigramas
        for cuatrigrama, freq in cuatrigramas.items():
            trigrama = (cuatrigrama[0], cuatrigrama[1], cuatrigrama[2])
            cuatrigramas_prob[cuatrigrama] = (freq + alpha) / (trigramas[trigrama] + V * alpha)  # P(w4|w1,w2,w3)
        return cuatrigramas_prob

    # FUNCIÓN PARA GENERAR K PALABRAS BASADO EN EL MODELO
    def predict_next_word_naive_bayes(self,w1, w2, w3, unigrams, bigrams, trigrams, cuatrigramas, alpha=0.1):
        # Lista de palabras candidatas (por simplicidad, usamos el vocabulario de unigramas)
        vocabulario = list(unigrams.keys())
        max_prob = 0
        best_word = None
        # Alamacenamos las probabilidades de cada palabra candidata
        probs = {}
        
        # Iterar por cada palabra candidata en el vocabulario
        for w4 in vocabulario:
            # Calcular las probabilidades individuales para cada n-grama
            # Cuatrigrama
            if (w1, w2, w3, w4) in cuatrigramas:
                p_cuatrigrama = (cuatrigramas[(w1, w2, w3, w4)] + alpha) / (trigrams[(w1, w2, w3)] + alpha * len(vocabulario))
            else:
                p_cuatrigrama = alpha / (trigrams.get((w1, w2, w3), 0) + alpha * len(vocabulario))

            # Trigrama
            if (w2, w3, w4) in trigrams:
                p_trigrama = (trigrams[(w2, w3, w4)] + alpha) / (bigrams[(w2, w3)] + alpha * len(vocabulario))
            else:
                p_trigrama = alpha / (bigrams.get((w2, w3), 0) + alpha * len(vocabulario))

            # Bigrama
            if (w3, w4) in bigrams:
                p_bigrama = (bigrams[(w3, w4)] + alpha) / (unigrams[w3] + alpha * len(vocabulario))
            else:
                p_bigrama = alpha / (unigrams.get(w3, 0) + alpha * len(vocabulario))

            # Unigrama (para suavizado en general)
            p_unigrama = (unigrams[w4] + alpha) / (sum(unigrams.values()) + alpha * len(vocabulario))

            # Calcular la probabilidad conjunta usando Naive Bayes (producto de las probabilidades)
            prob_total = p_cuatrigrama * p_trigrama * p_bigrama * p_unigrama
            #print(f'{w4} : {prob_total}')
            probs[w4] = prob_total

            # Mantener la palabra con la probabilidad más alta
            if prob_total > max_prob:
                max_prob = prob_total
                best_word = w4

        return best_word,probs

    def predict_words(self, frase: str, k: int, alpha=0.1):
        words = self.tokenizer(frase)
        n = len(words)
        if n < 3:
            return None
        w1 = words[-3]
        w2 = words[-2]
        w3 = words[-1]
        new_words = []
        for i in range(k):
            new_word, probs = self.predict_next_word_naive_bayes(w1, w2, w3, self.unigrams, self.bigrams, self.trigrams, self.cuatrigramas, alpha)

            # Ordenamos las probabilidades y seleccionamos el top 5
            top_5_probs_ord = sorted(probs.items(), key=lambda x: x[1], reverse=True)[:5]
            
            # Seleccionar una palabra aleatoria del top 5
            selected_word, selected_prob = top_5_probs_ord[np.random.randint(5)]
            
            # Imprimir la palabra predicha y su probabilidad
            print(f'{w1} {w2} {w3} {selected_word} : {selected_prob}')

            # Actualizar las palabras para la siguiente iteración
            w1 = w2
            w2 = w3
            w3 = selected_word

            new_words.append(selected_word)

        continuacion = ' '.join(new_words)
        return continuacion
    
    def calcular_entropia(self, test_text:str)->float:
        text_tokenized = self.tokenizer(test_text)
        total_words = len(text_tokenized)
        
        entropia = 0.0
        
        for i in range(total_words-3):
            cuatrigram = (text_tokenized[i],text_tokenized[i+1],text_tokenized[i+2],text_tokenized[i+3])
            trigram =(text_tokenized[i],text_tokenized[i+1],text_tokenized[i+2])
            bigram = (text_tokenized[i],text_tokenized[i+1])
            unigram = text_tokenized[i] 
            print(f'secuencia: {cuatrigram}')

            if cuatrigram in self.cuatrigrams_prob:
                p = self.cuatrigrams_prob[cuatrigram]                
                entropia += -math.log2(p)
                print(f'Se encontro cuatrigrama: {cuatrigram} [p:{p}] - [e:{entropia}]')
            elif trigram in self.trigrams_prob:
                
                p = self.trigrams_prob[trigram]
                entropia += -math.log2(p)
                print(f'Se encontro trigrama: {trigram} [p:{p}] - [e:{entropia}]')
            elif bigram in self.bigrams_prob:

                p = self.bigrams_prob[bigram]
                entropia += -math.log2(p)
                print(f'Se encontro bigrama: {bigram} [p:{p}] - [e:{entropia}]')
            elif unigram in self.unigrams_prob:
                p = self.unigrams_prob[unigram]
                entropia += -math.log2(p)
                print(f'Se encontro unigrama: {unigram} [p:{p}] - [e:{entropia}]')


        return entropia/total_words
    
    def calcular_perplejidad(self,test_text:str)->float:
        entropia = self.calcular_entropia(test_text)
        perplejidad = np.exp(entropia)
        print(f'entropia : {entropia}')
        print(f'Perplejidad (np.exp(entropia)): {perplejidad}')
        return perplejidad



In [155]:
# Ejemplo de uso:
path_corpus = './corpus/arthur-conan-doyle.tok.test.txt'
model = NaiveBayesNGramModel(path_corpus)


In [156]:
frase = 'the human is'
next_words = model.predict_words(frase=frase,k=12)
print(f'Frase predicha: [{frase}] {next_words}')

the human is to : 7.402095534383316e-12
human is to him : 1.4517975459305605e-11
is to him the : 6.29533609210717e-09
to him the young : 9.410074132252732e-13
him the young and : 1.548982979666425e-12
the young and to : 8.958372933314113e-12
young and to see : 2.978602786523119e-11
and to see you : 2.7280393569639796e-10
to see you have : 9.655515288715098e-12
see you have been : 2.515639709128347e-10
you have been in : 7.263334517401634e-11
have been in which : 5.933035310408693e-12
Frase predicha: [the human is] to him the young and to see you have been in which


### Cálculo de Entropía en un Modelo de Naive Bayes basado en N-gramas

#### Entropía

La **entropía** es una medida de la incertidumbre en la predicción de la siguiente palabra en una secuencia de texto. Cuanto mayor sea la entropía, más impredecible es la siguiente palabra, y viceversa. Se calcula como la suma del logaritmo negativo de las probabilidades condicionales de cada n-grama (cuatrigrama, trigrama, bigrama, unigrama) observado en el texto de prueba.

La fórmula de la entropía para un texto tokenizado se puede expresar de la siguiente manera:

$$
H = - \frac{1}{N} \sum_{i=1}^{N} \log_2 P(w_i \mid w_{i-3}, w_{i-2}, w_{i-1})
$$

Donde:

- \( $P(w_i \mid w_{i-3}, w_{i-2}, w_{i-1})$ \) es la probabilidad condicional de la palabra \( $w_i$ \) dada la historia de las tres palabras anteriores, basada en el modelo de cuatrigramas. Si no hay cuatrigramas disponibles, se utilizan trigramas, bigramas o unigramas.
- \( N \) es el número total de palabras en el texto de prueba.

El código implementado para calcular la entropía recorre el texto de prueba y, dependiendo de la disponibilidad de n-gramas, ajusta las probabilidades para las diferentes secuencias.

In [157]:
entropia = model.calcular_entropia('the direction of your life')


secuencia: ('the', 'direction', 'of', 'your')
Se encontro trigrama: ('the', 'direction', 'of') [p:0.0010202402501234163] - [e:9.936875361045546]
secuencia: ('direction', 'of', 'your', 'life')
Se encontro bigrama: ('direction', 'of') [p:0.005547602004294917] - [e:17.430795356068135]


In [158]:
print(f'entropia: {entropia}')

entropia: 3.486159071213627


### Cálculo de Perplejidad en un Modelo de Naive Bayes basado en N-gramas

#### Perplejidad

La **perplejidad** es una métrica utilizada para evaluar qué tan bien un modelo de lenguaje predice una secuencia de texto. Cuanto más baja es la perplejidad, mejor es el modelo para predecir las palabras siguientes. En otras palabras, la perplejidad es una medida de incertidumbre: indica cuántas palabras diferentes el modelo considera probables en promedio.

La perplejidad \( PP \) está relacionada directamente con la entropía \( H \), y se calcula de la siguiente manera:

$$
PP = 2^H
$$

O alternativamente:

$$
PP = e^H
$$

Donde:

- \( H \) es la entropía del modelo. Si la entropía es baja, significa que el modelo asigna probabilidades más altas a las palabras correctas, lo que da lugar a una perplejidad baja.
- Una perplejidad baja indica que el modelo de lenguaje es bueno para predecir la secuencia de palabras.



In [139]:
perplejidad = model.calcular_perplejidad('You build the direction of your life')

secuencia: ('you', 'build', 'the', 'direction')
Se encontro unigrama: you [p:0.012897232989764385] - [e:6.2767946109585155]
secuencia: ('build', 'the', 'direction', 'of')
secuencia: ('the', 'direction', 'of', 'your')
Se encontro trigrama: ('the', 'direction', 'of') [p:0.0010202402501234163] - [e:16.213669972004062]
secuencia: ('direction', 'of', 'your', 'life')
Se encontro bigrama: ('direction', 'of') [p:0.005547602004294917] - [e:23.70758996702665]
entropia : 3.3867985667180927
Perplejidad (np.exp(entropia)): 29.571130567096375


In [140]:
print(f'perplejidad: {perplejidad}')

perplejidad: 29.571130567096375


## Probando otros Corpus

In [159]:
# Ejemplo de uso:
path_corpus = './corpus/COSER-0102-01.txt'
model = NaiveBayesNGramModel(path_corpus)

In [160]:
print(model.corpus[:1500])

Provincia:	Álava
Enclave:	Berganzo (Zambrana) (0102)
Fecha:	14 de julio de 1992
Duración:	00:50:11:000
Informantes:	I1: mujer, 71 años
Encuesta:	José Ignacio Sanjuán, Reyes Fernández Sánchez, Teodora Fernández Olmo
Transcripción:	María José González Arévalo, César Fernández Ripoll, Aitana Cerviño
Temas:	2. Alimentación (pan, horno, manteca, huevos, bollos, recetas, pescado)
3. Animales domésticos (burro, caballo, buey, conejo, gallina, gato, perro)
6. Agricultura (agricultor, campo, cereal, legumbres, huerto, patata, viñas)
7. Ganadería (cabra, oveja, vaca, pastor, leche, queso, lana, abejas, miel)
8. Otras industrias (excluyendo agricultura y ganadería)
14. Bodas y noviazgos (boda, anillos, ajuar)
15. Fiestas populares (romerías, fiestas patronales, carnaval)
16. Sanidad y salud (enfermedad, médico, parto, entierro)
17. Construcciones (la casa, tipos, dependencias, infraestructuras)
20. Caza, pesca y monte (leña, animales y plantas no domésticos)
22. Tiempo meteorológico

I1: La merie

In [161]:
' '.join(model.corpus_tokenized)

'3 animales domésticos burro caballo buey conejo gallina gato perro 6 agricultura agricultor campo cereal legumbres huerto patata viñas 7 ganadería cabra oveja vaca pastor leche queso lana abejas miel 8 otras industrias excluyendo agricultura y ganadería 14 bodas y noviazgos boda anillos ajuar 15 fiestas populares romerías fiestas patronales carnaval 16 sanidad y salud enfermedad médico parto entierro 17 construcciones la casa tipos dependencias infraestructuras 20 caza pesca y monte leña animales y plantas no domésticos 22 tiempo meteorológico i1 la merienda es o o unos huevos fritos bueno bueno e1 t7 y y aparte del cerdo qué más animales tenían en casa i1 pues hombre pues había muchos los animales los que tenías para labrar los bueyes e1 bueyes para labrar i1 yeguas o caballos machos sí cerdos como siempre e1 y ah no tenían vacas para leche o i1 sí pero el que más tenía una pa la casa o no sé entonces bien e2 pero siempre ha habido aquí vacas en el pueblo i1 sí siempre hay había much

In [153]:
frase = 'cerdos ? ... que hacemos'
next_words = model.predict_words(frase=frase,k=20)

print(f'Frase predicha: [{frase}] {next_words}')

cerdos que hacemos y : 2.79751533591817e-11
que hacemos y y : 1.0201430093719854e-09
hacemos y y y : 1.00552626098232e-08
y y y a : 2.94772833165009e-08
y y a que : 1.9716877659464636e-08
y a que no : 1.140965544359159e-09
a que no se : 5.6520348565638604e-09
que no se le : 6.58968946476951e-09
no se le podía : 4.2031892796648823e-10
se le podía lavar : 4.1661812980820137e-10
le podía lavar i1 : 2.4081068976003e-08
podía lavar i1 no : 1.8172046151135987e-07
lavar i1 no no : 6.568227754914533e-07
i1 no no no : 9.242703515672144e-08
no no no no : 8.885754407014305e-09
no no no no : 8.885754407014305e-09
no no no no : 8.885754407014305e-09
no no no no : 8.885754407014305e-09
no no no no : 8.885754407014305e-09
no no no no : 8.885754407014305e-09
Frase predicha: [cerdos ? ... que hacemos] y y y a que no se le podía lavar i1 no no no no no no no no no


### Corrección de la predicción de la siguiente palabra usando el top 5 de probabilidades

En la función `predict_words`, para evitar que siempre se seleccione la palabra con mayor probabilidad, se implementó una selección aleatoria entre las 5 palabras con las probabilidades más altas. 

#### Ajustes realizados:
1. **Ordenar y seleccionar el top 5**:
   ```python
   top_5_probs_ord = sorted(probs.items(), key=lambda x: x[1], reverse=True)[:5]


In [162]:
frase = 'cerdos ? ... que hacemos'
next_words = model.predict_words(frase=frase,k=20)

print(f'Frase predicha: [{frase}] {next_words}')

cerdos que hacemos que : 1.843725074033327e-11
que hacemos que se : 1.042597876552835e-09
hacemos que se iba : 2.3146665819730145e-10
que se iba con : 6.478361595908353e-10
se iba con y : 1.847987995299701e-10
iba con y y : 1.0117605688676961e-09
con y y a : 2.7019547937170744e-09
y y a hacer : 4.576809568464352e-10
y a hacer eso : 4.94390702428132e-10
a hacer eso y : 3.838640996690733e-10
hacer eso y nada : 2.7802442877005545e-10
eso y nada aquí : 2.9601154246047327e-10
y nada aquí a : 1.829860065668699e-10
nada aquí a que : 1.7367267472298898e-10
aquí a que se : 1.025603616136326e-09
a que se hacían : 4.508621583538604e-10
que se hacían la : 2.2929333693383317e-10
se hacían la era : 9.716719739558427e-11
hacían la era pues : 6.629643087966012e-10
la era pues a : 2.5120472792131404e-10
Frase predicha: [cerdos ? ... que hacemos] que se iba con y y a hacer eso y nada aquí a que se hacían la era pues a


# Conclusión

Se pudo arreglar de cierta manera la repetición o bucle de palabras, sin embargo el predictor sigue siendo un tanto carente de sentido contextual. Por solo usar una ventana de 4 palabras. Usar más tambien haria que se hiciera un calculo muy grande por ello no es recomendable seguir espandiendo el Naive Bayes.Se recomienda migrar hacia enfoques más avanzados como el uso de word embeddings, los cuales capturan relaciones semánticas entre palabras en un espacio de alta dimensionalidad.